#This notebook is to collect the images

In [ ]:
import pandas as pd

# 1. Load your filtered Nepal Landmark IDs
nepal_ids_df = pd.read_csv('nepal_monuments.csv')
nepal_list = nepal_ids_df['landmark_id'].tolist()

# 2. Load the main train.csv (Using 'usecols' saves memory)
df_train = pd.read_csv('../datasets/train.csv', usecols=['id', 'url', 'landmark_id'])

# 3. Filter for only Nepal images
nepal_images = df_train[df_train['landmark_id'].isin(nepal_list)]

# 4. Save this specific list
nepal_images.to_csv('nepal_images_to_download.csv', index=False)
print(f"Ready to download {len(nepal_images)} images!")

Ready to download 2774 images!


In [ ]:
import os
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm  # This adds a progress bar

# Load your filtered list
df = pd.read_csv('nepal_images_to_download.csv')

# a 'Session' to make downloading faster and more reliable
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
})

def download_image(row):
    image_id = row['id']
    url = row['url']
    l_id = str(row['landmark_id'])

    folder_path = os.path.join('nepal_dataset', l_id)
    os.makedirs(folder_path, exist_ok=True)

    file_path = os.path.join(folder_path, f"{image_id}.jpg")

    if os.path.exists(file_path):
        return

    try:
        # We add a timeout and allow redirects
        response = session.get(url, timeout=15, allow_redirects=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
        else:
            # This will help to see WHY it's failing
            print(f" Failed: {image_id} (Status: {response.status_code})")
    except Exception as e:
        pass # Ignore errors to keep the progress bar clean

# Wrap the executor in a progress bar
with ThreadPoolExecutor(max_workers=10) as executor:
    list(tqdm(executor.map(download_image, [row for _, row in df.iterrows()]),
              total=len(df), desc="Downloading Nepal Monuments"))

print("\nDownload process finished! Check your 'nepal_dataset' folder now.")

 Failed: 58a868a053ed29aa (Status: 404)


 Failed: 23548a412c816eef (Status: 404)


 Failed: aa2d6d823cb40bff (Status: 404)


 Failed: 0e811d85e89162cf (Status: 404)


 Failed: 9ee29dc7a4b62943 (Status: 404)


...

 Failed: c6dae8e80b91f46d (Status: 404)
 Failed: ae44e114ac2bc927 (Status: 404)


 Failed: 6f107103599ff646 (Status: 404)


 Failed: b345dab4b13a59a8 (Status: 404)


 Failed: e62068d89c6c3dbd (Status: 404)



Download process finished! Check your 'nepal_dataset' folder now.


In [ ]:
import os
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Load your filtered list
df = pd.read_csv('nepal_images_to_download.csv')

def download_image(row):
    image_id = row['id']
    url = row['url']
    l_id = str(row['landmark_id'])

    # Create a folder for each landmark if it doesn't exist
    folder_path = os.path.join('nepal_dataset', l_id)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path, exist_ok=True)

    file_path = os.path.join(folder_path, f"{image_id}.jpg")

    # Skip if already downloaded
    if os.path.exists(file_path):
        return

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"Error downloading {image_id}: {e}")

# ThreadPool to speed up the process (adjust max_workers based on your internet)
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(download_image, [row for _, row in df.iterrows()])

print("Download complete!")